In [543]:
import pandas as pd
import numpy as np
import datetime
now = datetime.datetime.now()

In [544]:
from sqlalchemy import create_engine

In [545]:
sqlitedb_path = '/home/miguelmaragon/Documentos/IRONHACK/Ironhack-Module-1-Project---Pipelines---MMA/data/raw/mmaragon.db'

In [546]:
engine = create_engine(f'sqlite:///{sqlitedb_path}')

In [547]:
df_sqlite_b_info = pd.read_sql_query("select * from business_info", engine)
df_sqlite_p_info = pd.read_sql_query("select * from personal_info", engine)
df_sqlite_r_info = pd.read_sql_query("select * from rank_info", engine)

In [548]:
df_sqlite_b_info = df_sqlite_b_info.drop(["realTimeWorth", "Unnamed: 0"], axis=1)

In [549]:
df_sqlite_p_info = df_sqlite_p_info.drop(["Unnamed: 0"], axis=1)

In [550]:
def title_COLUMN(df, column):
    df[column] = df[column].str.title()
    return df[column]
title_COLUMN(df_sqlite_p_info, 'lastName')

0            Bezos
1            Gates
2          Buffett
3          Arnault
4       Zuckerberg
           ...    
2203          Zhao
2204          Zhou
2205           Zhu
2206          Zhuo
2207         Zorlu
Name: lastName, Length: 2208, dtype: object

In [551]:
df_sqlite_r_info = df_sqlite_r_info.drop(["Unnamed: 0"], axis=1)

In [552]:
title_COLUMN(df_sqlite_r_info, 'name')

0       Anant Asavabhokhin & Family
1                    Arash Ferdowsi
2          Banthoon Lamsam & Family
3            Bill Adderley & Family
4                 Boris Zingarevich
                   ...             
2255                 Zhao Xiaoqiang
2256                Zhou Liangzhang
2257                   Zhu Xingming
2258                       Zhuo Jun
2259                    Olgun Zorlu
Name: name, Length: 2260, dtype: object

In [553]:
df_sqlite_b_info.isnull().sum()

id                  0
Source              0
worth               0
worthChange         0
realTimePosition    0
dtype: int64

In [554]:
df_sqlite_p_info.isnull().sum()

id           0
lastName     0
age         65
gender      13
country      0
image        0
dtype: int64

In [555]:
df_sqlite_r_info.isnull().sum()

id           0
name         0
position    52
dtype: int64

In [556]:
from functools import reduce
dfs = [df_sqlite_b_info, df_sqlite_p_info, df_sqlite_r_info]
df_final = reduce(lambda left,right: pd.merge(left,right,on='id'), dfs)

In [557]:
df_final = df_final[['id', 'position','realTimePosition', 'name', 'lastName', 'age','gender','country','Source','worth','worthChange', 'image']]
df_final.rename(columns = {'id':'Id','position':'Position','realTimePosition':'RealTimePosition','name':'Name','lastName':'LastName','age':'Age','gender':'Gender','country':'Country','worth':'Worth','worthChange':'WorthChange','image':'Image'}, inplace = True)

In [558]:
df_final.Position = df_final.Position.astype(int)
df_final = df_final.sort_values(by='Position', ascending=True)

In [559]:
df_final

,Id,Position,RealTimePosition,Name,LastName,Age,Gender,Country,Source,Worth,WorthChange,Image
0,5110,1,1,Jeff Bezos,Bezos,54 years old,M,United States,Technology ==> Amazon,112.0 BUSD,0.0 millions USD,https://specials-images.forbesimg.com/imageser...
1,8029,2,2,Bill Gates,Gates,62 years old,None,United States,Technology ==> Microsoft,90.0 BUSD,-0.001 millions USD,https://specials-images.forbesimg.com/imageser...
2,1017,3,3,Warren Buffett,Buffett,87 years old,None,United States,Finance and Investments ==> Berkshire Hathaway,84.0 BUSD,-0.002 millions USD,https://specials-images.forbesimg.com/imageser...
3,3437,4,4,Bernard Arnault,Arnault,69 years old,None,None,Fashion & Retail ==> LVMH,72.0 BUSD,0.0 millions USD,https://specials-images.forbesimg.com/imageser...
4,4589,5,5,Mark Zuckerberg,Zuckerberg,1985,None,United States,Technology ==> Facebook,71.0 BUSD,0.0 millions USD,https://specials-images.forbesimg.com/imageser...
...,...,...,...,...,...,...,...,...,...,...,...,...
2203,1536,2204,2134,Zhao Xiaoqiang,Zhao,51 years old,M,China,"Fashion & Retail ==> fashion, entertainment",1.0 BUSD,0.0 millions USD,https://specials-images.forbesimg.com/imageser...
2204,3722,2205,2134,Zhou Liangzhang,Zhou,55 years old,M,None,Manufacturing ==> electrical equipment,1.0 BUSD,nan millions USD,https://specials-images.forbesimg.com/imageser...
2205,7308,2206,1856,Zhu Xingming,Zhu,51 years old,M,China,Manufacturing ==> electrical equipment,1.0 BUSD,0.0 millions USD,https://specials-images.forbesimg.com/imageser...
2206,7816,2207,1978,Zhuo Jun,Zhuo,52 years old,None,Hong Kong,Manufacturing ==> printed circuit boards,1.0 BUSD,0.0 millions USD,https://specials-images.forbesimg.com/imageser...


In [560]:
def Gender_f(the_row):
    if the_row == "M":
        return "Male"
    elif the_row == "F":
        return "Female"
    else:
        return the_row
def Age_f(the_row):
    if the_row >= 1000:
        return now.year - the_row
    else:
        return the_row

In [561]:
df_final['Gender'] = df_final.apply(lambda row: Gender_f(row.Gender), axis=1)

df_final['Age'] = df_final.Age.str.replace(' years old', '')
df_final['Age'] = df_final.Age.fillna(0)
df_final['Age'] = df_final.Age.astype(int)
df_final['Age'] = df_final.apply(lambda row: Age_f(row.Age), axis=1)

df_final['Worth'] = df_final.Worth.str.replace(' BUSD', '')
df_final['Worth'] = df_final.Worth.astype(float)

df_final['WorthChange'] = df_final.WorthChange.str.replace(' millions USD', '')
df_final['WorthChange'] = df_final.WorthChange.astype(float)

df_final['Sector'] = df_final.Source.str.split(' ==> ').str[0].str.title()
df_final['Company'] = df_final.Source.str.split(' ==> ').str[1].str.title()
df_final = df_final.drop(["Source"], axis=1)

In [562]:
df_final.head(50)

,Id,Position,RealTimePosition,Name,LastName,Age,Gender,Country,Worth,WorthChange,Image,Sector,Company
0,5110,1,1,Jeff Bezos,Bezos,54,Male,United States,112.0,0.000,https://specials-images.forbesimg.com/imageser...,Technology,Amazon
1,8029,2,2,Bill Gates,Gates,62,None,United States,90.0,-0.001,https://specials-images.forbesimg.com/imageser...,Technology,Microsoft
2,1017,3,3,Warren Buffett,Buffett,87,None,United States,84.0,-0.002,https://specials-images.forbesimg.com/imageser...,Finance And Investments,Berkshire Hathaway
3,3437,4,4,Bernard Arnault,Arnault,69,None,None,72.0,0.000,https://specials-images.forbesimg.com/imageser...,Fashion & Retail,Lvmh
4,4589,5,5,Mark Zuckerberg,Zuckerberg,34,None,United States,71.0,0.000,https://specials-images.forbesimg.com/imageser...,Technology,Facebook
5,1386,6,6,Amancio Ortega,Ortega,82,Male,None,70.0,0.000,https://specials-images.forbesimg.com/imageser...,Fashion & Retail,Zara
6,8213,7,7,Carlos Slim Helu,Slim Helu,78,Male,None,67.1,0.001,https://specials-images.forbesimg.com/imageser...,Telecom,Telecom
7,1939,8,12,Charles Koch,Koch,82,Male,None,60.0,0.000,https://specials-images.forbesimg.com/imageser...,Diversified,Koch Industries
8,5086,9,13,David Koch,Koch,78,Male,United States,60.0,0.000,https://specials-images.forbesimg.com/imageser...,Diversified,Koch Industries
9,6729,10,8,Larry Ellison,Ellison,73,Male,None,58.5,-0.001,https://specials-images.forbesimg.com/imageser...,Technology,Software


In [566]:
df_final.to_csv('../data/processed/ForbesBillionaires2018.csv', index=False)

In [563]:
df_final[['Id','Age']].groupby(['Age']).count().sort_values(by='Age', ascending=True)

,Id
Age,
0,65
21,1
23,1
25,1
27,1
...,...
94,7
95,4
96,1
